In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [1]:
#https://drive.google.com/file/d/1r1JdGH2YLDWkIAw8gWpwPXDqrBfsJDR_/view?usp=drive_link

!gdown '1r1JdGH2YLDWkIAw8gWpwPXDqrBfsJDR_'

Downloading...
From (original): https://drive.google.com/uc?id=1r1JdGH2YLDWkIAw8gWpwPXDqrBfsJDR_
From (redirected): https://drive.google.com/uc?id=1r1JdGH2YLDWkIAw8gWpwPXDqrBfsJDR_&confirm=t&uuid=1ecd6f36-ea5d-4fd1-bc08-341281800fbc
To: /kaggle/working/training_논문.zip
100%|████████████████████████████████████████| 164M/164M [00:03<00:00, 51.5MB/s]


In [2]:
import os
print(os.listdir("/kaggle/working"))

['training_논문.zip', '.virtual_documents']


In [3]:
import zipfile

# 파일 경로 설정
zip_path = '/kaggle/working/training_논문.zip'
directory_to_extract = '/kaggle/working/'

# 압축 해제
with zipfile.ZipFile(zip_path, 'r') as zip_ref:
    for file in zip_ref.namelist():
        # 파일 이름 복구 (cp437에서 euc-kr로 변환)
        decoded_file_name = file.encode('cp437').decode('euc-kr')

        # 압축 해제 후, 복구된 파일명으로 저장
        zip_ref.extract(file, directory_to_extract)

        # 파일 경로 수정
        original_path = os.path.join(directory_to_extract, file)
        new_path = os.path.join(directory_to_extract, decoded_file_name)

        # 파일 이름 변경 (복구된 파일명으로)
        if os.path.exists(original_path):
            os.rename(original_path, new_path)

In [4]:
import shutil
shutil.rmtree('/kaggle/working/training_│φ╣«/')

In [5]:
j = ['논문요약20231006_3.json',
 '논문요약20231006_0.json',
 '논문요약20231006_1.json',
 '논문요약20231006_2.json']

In [8]:
import json
import pandas as pd
# 빈 리스트 초기화
original_text = []  # 원문 리스트
original_summary = []  # 원문 요약 리스트
section_text = []  # 섹션 원문 리스트
section_summary = []  # 섹션 요약 리스트

# 파일 리스트 j (파일 리스트를 정의해야 합니다)
for file in j:
    with open(f'/kaggle/working/training_논문/{file}', 'r', encoding='utf-8') as f:
        data = json.load(f)

        # 데이터에서 각 항목 추출
        for i in range(len(data)):
            # 전체 원문과 요약
            df = pd.DataFrame(data[i]['data'])

            # 전체 원문과 전체 요약 추출
            df['original_text'] = df['summary_entire'].apply(lambda x: x[0]['orginal_text'])  # 전체 원문
            df['original_summary'] = df['summary_entire'].apply(lambda x: x[0]['summary_text'])  # 원문 요약

            # 섹션 원문과 섹션 요약
            df['section_text'] = df['summary_section'].apply(lambda x: x[0]['orginal_text'])  # 섹션 원문
            df['section_summary'] = df['summary_section'].apply(lambda x: x[0]['summary_text'])  # 섹션 요약

            # 원문과 원문 요약을 리스트에 추가
            original_text.extend(df['original_text'].values)
            original_summary.extend(df['original_summary'].values)

            # 섹션 원문과 섹션 요약을 리스트에 추가
            section_text.extend(df['section_text'].values)
            section_summary.extend(df['section_summary'].values)

# 1. 데이터프레임에 source_type 추가
original_df = pd.DataFrame({
    'original_text': original_text,
    'summary_text': original_summary,
    'source_type': ['original'] * len(original_text)
})
section_df = pd.DataFrame({
    'original_text': section_text,
    'summary_text': section_summary,
    'source_type': ['section'] * len(section_text)
})

# 2. 통합 및 셔플
merged_df = pd.concat([original_df, section_df]).sample(frac=1).reset_index(drop=True)

# 3. 프롬프트 추가
def add_prompt(row):
    if row["source_type"] == "original":
        return "[전체요약] " + row["original_text"]
    else:
        return "[섹션요약] " + row["original_text"]

merged_df["input_text"] = merged_df.apply(add_prompt, axis=1)

In [9]:
merged_df

,original_text,summary_text,source_type,input_text
0,본 연구의 목적은 초등교사들이 인식하는 감염병 재난대응역량의 특성이 무엇인지를 밝히...,"본 연구는 초등교사들이 인식하는 감염병 재난대응 역량의 특성이 무엇인지를 규명하고,...",original,[전체요약] 본 연구의 목적은 초등교사들이 인식하는 감염병 재난대응역량의 특성이 무...
1,본 연구는 체육 영재 양성 프로그램에 참여한 남녀 초등학생의 체격상태를 고려한 체력...,본 연구의 목적은 체육 영재 양성 프로그램에 참여한 남녀 초등생의 체격상태를 고려한...,original,[전체요약] 본 연구는 체육 영재 양성 프로그램에 참여한 남녀 초등학생의 체격상태를...
2,본 연구는 스포츠 현장에서 싱크로나이즈드 스위밍의 심판이 공정하고 일관성 있는 판정...,본고는 스포츠 현장에서 싱크로나이즈드 스위밍의 심판이 공정하고 일관성 있는 판정 가...,original,[전체요약] 본 연구는 스포츠 현장에서 싱크로나이즈드 스위밍의 심판이 공정하고 일관...
3,최근 보건복지부(2005) 연구보고에 따르면 20대의 비만인구가 1992년 8.1%...,여자 대학생의 불안 심리와 부적절한 식행동은 밀접한 연관성을 가지며 이는 비만을 유...,section,[섹션요약] 최근 보건복지부(2005) 연구보고에 따르면 20대의 비만인구가 199...
4,"본 연구는 MOOC 운영의 성과를 진단하고, 학습자 특성별 운영방향을 제안하기 위해...","본 연구는 MOOC 운영의 성과를 진단하고, 학습자 특성별 운영방향을 제안하기 위해...",original,"[전체요약] 본 연구는 MOOC 운영의 성과를 진단하고, 학습자 특성별 운영방향을 ..."
...,...,...,...,...
288169,본 연구는 우리나라 대도시의 도심쇠퇴를 다루고 있다. 연구의 목적은 다양한 사회·경...,분산 및 공동화지표의 구축과 설명계수의 비교분석 결과 분산지표에 따라 분산지표 감소...,original,[전체요약] 본 연구는 우리나라 대도시의 도심쇠퇴를 다루고 있다. 연구의 목적은 다...
288170,본 연구는 북아프리카의 서쪽인 마그레브(Maghreb)지역의 베르베르(Berber)...,본고는 북아프리카의 베르베르(Berber)원주민들의 혼성문화 형성 과정을 고찰하고 ...,original,[전체요약] 본 연구는 북아프리카의 서쪽인 마그레브(Maghreb)지역의 베르베르(...
288171,현장에서 파고와 주기만을 관측한 경우 주로 개별파 분석법을 사용하여 다양한 파랑 특...,현장에서 파고와 주기만을 관측한 경우 개별파 분석법을 사용해 다양한 파랑 특성을 산...,original,[전체요약] 현장에서 파고와 주기만을 관측한 경우 주로 개별파 분석법을 사용하여 다...
288172,조석에 의한 흐름이 지배적인 해역에서 조석 유입구(tidalentrance)는 해수...,조석 유입구에서는 유입되는 유량과 유출되는 유량이 균형을 이루게 되면 해수와 함께 ...,section,[섹션요약] 조석에 의한 흐름이 지배적인 해역에서 조석 유입구(tidalentran...


In [10]:
from datasets import Dataset

dataset = Dataset.from_pandas(merged_df)

In [11]:
dataset

Dataset({
    features: ['original_text', 'summary_text', 'source_type', 'input_text'],
    num_rows: 288174
})

In [12]:
!git clone https://github.com/SKT-AI/KoBART #egg=kobart
%cd /kaggle/working/KoBART
os.getcwd()

Cloning into 'KoBART'...
remote: Enumerating objects: 334, done.
remote: Counting objects: 100% (31/31), done.
remote: Compressing objects: 100% (18/18), done.
remote: Total 334 (delta 15), reused 13 (delta 13), pack-reused 303 (from 1)
Receiving objects: 100% (334/334), 8.24 MiB | 21.25 MiB/s, done.
Resolving deltas: 100% (178/178), done.
/kaggle/working/KoBART


'/kaggle/working/KoBART'

In [13]:
from kobart import get_kobart_tokenizer,get_pytorch_kobart_model
#from transformers import BartForConditionalGeneration#,BartTokenizer
tokenizer = get_kobart_tokenizer()

/kaggle/working/KoBART/.cache/kobart_base_tokenizer_cased_cf74400bce.zip[██████████████████████████████████████████████████]


In [14]:
def tokenize_function(examples):
    # 프롬프트가 붙은 input_text를 인코딩
    model_inputs = tokenizer(
        examples["input_text"],
        max_length=512,
        padding="max_length",
        truncation=True
    )

    # 요약문도 target으로 인코딩
    with tokenizer.as_target_tokenizer():
        labels = tokenizer(
            examples["summary_text"],
            max_length=128,
            padding="max_length",
            truncation=True
        )["input_ids"]

    # pad token은 loss 계산에서 무시되도록 -100으로 변경
    labels = [
        [(token if token != tokenizer.pad_token_id else -100) for token in label]
        for label in labels
    ]

    model_inputs["labels"] = labels
    return model_inputs

tokenized_dataset = dataset.map(tokenize_function, batched=True)

Map:   0%|          | 0/288174 [00:00<?, ? examples/s]

/usr/local/lib/python3.11/dist-packages/transformers/tokenization_utils_base.py:3980: UserWarning: `as_target_tokenizer` is deprecated and will be removed in v5 of Transformers. You can tokenize your labels by using the argument `text_target` of the regular `__call__` method (either in the same call as your input texts if you use the same keyword arguments, or in a separate call.
  warnings.warn(


In [16]:
tokenized_dataset

Dataset({
    features: ['original_text', 'summary_text', 'source_type', 'input_text', 'input_ids', 'token_type_ids', 'attention_mask', 'labels'],
    num_rows: 288174
})

In [18]:
tokenized_datasets = tokenized_dataset.remove_columns(['token_type_ids']);tokenized_datasets

Dataset({
    features: ['original_text', 'summary_text', 'source_type', 'input_text', 'input_ids', 'attention_mask', 'labels'],
    num_rows: 288174
})

In [19]:
tokenized_datasets

Dataset({
    features: ['original_text', 'summary_text', 'source_type', 'input_text', 'input_ids', 'attention_mask', 'labels'],
    num_rows: 288174
})

In [23]:
tokenized_dataset.save_to_disk("/kaggle/working/tokenized_datasets")

Saving the dataset (0/5 shards):   0%|          | 0/288174 [00:00<?, ? examples/s]

In [24]:
os.listdir('/kaggle/working')

['tokenized_datasets']